<a href="https://colab.research.google.com/github/VishalPrem1994/AIGenPlayGround/blob/main/TodayInHistoryGen/StaticImageGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Base Installations

In [ ]:
%pip install openai numpy opencv-python

In [1]:
import openai
import datetime

##### Configs

In [11]:
OPEN_AI_ORG = "" #@param {type:"string"}
OPEN_AI_KEY = "" #@param {type:"string"}
HUGGING_FACE_KEY = ""


openai.organization = OPEN_AI_ORG
openai.api_key = OPEN_AI_KEY

#### Functions

In [ ]:
class Image:
    def __init__(self, filename, time=500, size=500):
        self.size = size
        self.time = time
        self.shifted = 0.0
        self.img = cv2.imread(filename)
        self.height, self.width, _ = self.img.shape
        if self.width < self.height:
            self.height = int(self.height*size/self.width)
            self.width = size
            self.img = cv2.resize(self.img, (self.width, self.height))
            self.shift = self.height - size
            self.shift_height = True
        else:
            self.width = int(self.width*size/self.height)
            self.height = size
            self.shift = self.width - size
            self.img = cv2.resize(self.img, (self.width, self.height))
            self.shift_height = False
        self.delta_shift = self.shift/self.time
    def reset(self):
        if random.randint(0, 1) == 0:
            self.shifted = 0.0
            self.delta_shift = abs(self.delta_shift)
        else:
            self.shifted = self.shift
            self.delta_shift = -abs(self.delta_shift)
    def get_frame(self):
        if self.shift_height:
            roi = self.img[int(self.shifted):int(self.shifted) + self.size, :, :]
        else:
            roi = self.img[:, int(self.shifted):int(self.shifted) + self.size, :]
        self.shifted += self.delta_shift
        if self.shifted > self.shift:
            self.shifted = self.shift
        if self.shifted < 0:
            self.shifted = 0
        return roi

In [ ]:
def promptsForDate(date,promptType):
    if promptType=="IN":
        return "Summarize two famous historical events that happened on "+str(date.strftime("%d"))+" "+str(date.strftime("%B"))+" In India In history"
    else:
        return "Summarize two famous event that happened on "+str(date.strftime("%d"))+" "+str(date.strftime("%B"))

def promptForImageGeneration(rawText):
    return "Generate three detailed Prompts for giving ai art generation models for the following text "+str(rawText)

def promptsListCleanUp(rawTextList):
    newList = list()
    for i in rawTextList:
        if(len(i)!=0):
            i.translate(str.maketrans('', '', string.punctuation))
            newList.append(i)
    return newList

def getDaVinciResponse(prompt):
    promptResponse = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        temperature=0.7,
        max_tokens=200,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=1
    )
    promptResponse = promptResponse['choices'][0]['text'].split("\n\n")
    promptResponse = promptsListCleanUp(promptResponse)
    return promptResponse


In [ ]:
def process(path):
    filenames = glob.glob(os.path.join(path, "*"))
    cnt = 0
    images = []
    for filename in filenames:
        print(filename)
        img = Image(filename)
        images.append(img)
        if cnt > 300:
            break
        cnt += 1
    prev_image = images[random.randrange(0, len(images))]
    prev_image.reset()
    while True:
        while True:
            img = images[random.randrange(0, len(images))]
            if img != prev_image:
                break
        img.reset()
        for i in range(100):
            alpha = i/100
            beta = 1.0 - alpha
            dst = cv2.addWeighted(img.get_frame(), alpha, prev_image.get_frame(), beta, 0.0)
            cv2.imshow("Slide", dst)
            if cv2.waitKey(1) == ord('q'):
                return
        prev_image = img
        for _ in range(300):
            cv2.imshow("Slide", img.get_frame())
            if cv2.waitKey(1) == ord('q'):
                return

#### Main Code

In [12]:
promptResponse = getDaVinciResponse(promptsForDate(datetime.date.today(),"IN"))
promptResponse

["1. The Jallianwala Bagh Massacre: On April 13, 1919, British troops opened fire on an unarmed crowd of protesters in Amritsar, India, killing hundreds and wounding thousands in what is now known as the Jallianwala Bagh massacre. The incident was a major factor in the growing Indian independence movement and helped shape the country's future.",
 '2. The Indian Rebellion of 1857: On April 10, 1857, Indian troops stationed in the city of Meerut mutinied against the British East India Company and sparked a widespread rebellion that would become known as the Indian Rebellion of 1857. This uprising would eventually be suppressed by the British, but it marked a turning point in the history of India, leading to increased autonomy for the region and paving the way for independence from Britain.']

In [13]:
imagePrompts = [getDaVinciResponse(promptForImageGeneration(i)) for i in promptResponse] 
imagePrompts

[['1. Create a generative AI art piece that conveys a sense of sorrow and tragedy, while also implying the strength of the Indian people in the face of injustice. Focus on the colors of India’s flag, or incorporate the use of dark and light hues to represent the contrast between the horror of the massacre and the hope of the independence movement. ',
  '2. Create an AI art piece that illustrates the underlying tension between the British and Indian people before the massacre occurred, using symbolism and muted colors to capture the emotions of the time. Consider including images of the Amritsar cityscape as well as figures representing both sides of the conflict. ',
  '3. Create an AI art piece that gives a modern interpretation of the Jallianwala Bagh massacre. Incorporate abstract shapes and textures to symbolize the violence and chaos of the event, as well as bright colors to represent the resilience of the Indian people in the aftermath.'],
 ['1. Create an AI art generation model t

In [ ]:
process("Sample Images")